 In this example we test the capabilities of **ReactiveMP.jl** to run smoothing algorithm for large-scale datastream.

In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [2]:
@model function smoothing(n, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    c = constvar(1.0)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

In [5]:
P = 1.0

n = 10_000
k = 500
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [9]:
@benchmark res = inference(
        model=Model(smoothing, $n, NormalMeanVariance(0.0, 10000.0), $P), 
        data=(y=$data,), 
        options=model_options(limit_stack_depth = $k)
)

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  471.926 ms … 702.137 ms  ┊ GC (min … max): 18.22% … 47.59%
 Time  (median):     582.593 ms               ┊ GC (median):    33.76%
 Time  (mean ± σ):   583.198 ms ±  61.142 ms  ┊ GC (mean ± σ):  33.68% ±  8.11%

  ▁                  ▁     █   ▁ ▁  ▁   ▁                     ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁█▁█▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  472 ms           Histogram: frequency by time          702 ms <

 Memory estimate: 232.15 MiB, allocs estimate: 4291121.